In [1]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [ ]:
#=nb

In [3]:
class Alexnet(nn.Module):
  def __init__(self,):
    super().__init__()
    self.flatten = nn.Flatten()
    self.Conv = nn.Sequential(
        # 1st conv layer
        nn.Conv2d(in_channels = 3 , out_channels = 96, kernel_size = 11, stride = 4, padding = 2),
        nn.ReLU(),
        nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75, k = 2),
        nn.MaxPool2d(stride =2, kernel_size = 3),
        # 2nd conv layer
        nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5, stride =1, padding = 2),
        nn.ReLU(),
        nn.LocalResponseNorm(size =5, alpha = 0.0001, beta = 0.75, k = 2 ),
        nn.MaxPool2d(stride = 2, kernel_size = 3),
        # 3rd conv layer
        nn.Conv2d(in_channels = 256, out_channels = 384, stride = 1, padding = 1, kernel_size = 3),
        nn.ReLU(),
        #4th conv layer
        nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3, padding = 1, stride = 1),
        nn.ReLU(),
        #5th conv layer
        nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3, padding = 1, stride = 1),
        nn.ReLU(),
        nn.MaxPool2d(stride = 2, kernel_size = 2)
    )
    self.Fc = nn.Sequential(
        nn.Dropout(0,5),
        nn.Linear(256 * 6 * 6, 4096),
        nn.ReLU(),
        nn.Dropout(0,5),
        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Linear(4096, 10)
    )
    self.init_bias()
  def init_bias(self):
    for layer in self.Conv:
      if isinstance(layer, nn.Conv2d):
        nn.init.normal_(layer.weight, mean = 0, std = 0.01)
        nn.init.constant_(layer.bias, 0)
    nn.init.constant_(self.Conv[4].bias, 1)
    nn.init.constant_(self.Conv[10].bias, 1)
    nn.init.constant_(self.Conv[12].bias, 1)
  def forward(self, X):
    X = self.Conv(X)
    X = self.flatten(X)
    X = self.Fc(X)
    return X